# Generators, Iterators, and Asynchronous Programming

Les générateurs sont une autre de ces fonctionnalités qui font de Python un langage particulier par rapport aux langages plus traditionnels. Dans ce guide, nous explorerons leur raison d'être, pourquoi ils ont été introduits dans la langue et les problèmes qu'ils résolvent. Nous verrons également comment résoudre les problèmes de manière idiomatique en utilisant des générateurs et comment rendre nos générateurs (ou tout autre itérable, d'ailleurs) Pythonic.

On comprendra pourquoi l'itération (sous la forme du pattern itérateur) est automatiquement supportée dans le langage. À partir de là, nous ferons un autre voyage et explorerons comment les générateurs sont devenus une fonctionnalité si fondamentale de Python afin de prendre en charge d'autres fonctionnalités, telles que les coroutines et la programmation asynchrone.

Les objectifs de ce chapitre sont les suivants :

* Créer des générateurs qui améliorent les performances de nos programmes
*  Étudier comment les itérateurs (et le modèle d'itérateur, en particulier) sont profondément intégrés à Python

*  Résoudre les problèmes impliquant une itération de manière idiomatique

*  Comprendre comment les générateurs fonctionnent comme base pour les coroutines et la programmation asynchrone 
* Pour explorer la prise en charge syntaxique des coroutines :yield from, await,  et async def


La maîtrise des générateurs vous mènera loin dans l'écriture de Python idiomatique, d'où leur importance pour ce guide. Dans ce guide, nous étudions non seulement comment utiliser les générateurs, mais nous explorons également leurs internes, afin de comprendre en profondeur leur fonctionnement.

## Création de générateurs

Les générateurs ont été introduits en Python il y a longtemps (PEP-255), avec l'idée d'introduire l'itération en Python tout en améliorant les performances du programme (en utilisant moins de mémoire) en même temps.

L'idée d'un générateur est de créer un objet qui est itérable, et, pendant qu'il est itéré, produira les éléments qu'il contient, un à la fois. L'utilisation principale des générateurs est d'économiser de la mémoire - au lieu d'avoir une très grande liste d'éléments en mémoire, contenant tout à la fois, nous avons un objet qui sait comment produire chaque élément particulier, un à la fois, selon les besoins.

Cette fonctionnalité permet des calculs paresseux d'objets lourds en mémoire, d'une manière similaire à ce que fournissent d'autres langages de programmation fonctionnels (Haskell, par exemple). Il serait même possible de travailler avec des séquences infinies car la nature paresseuse des générateurs permet une telle option.



## Un premier regard sur les générateurs
Commençons par un exemple. Le problème actuel est que nous voulons traiter une grande liste d'enregistrements et obtenir des métriques et des indicateurs dessus. Étant donné un grand ensemble de données contenant des informations sur les achats, nous souhaitons le traiter afin d'obtenir la vente la plus basse, la vente la plus élevée et le prix moyen d'une vente.

Pour la simplicité de cet exemple, nous supposerons un CSV avec seulement deux champs, au format suivant :

    <purchase_date>, <price>


Nous allons créer un objet qui reçoit tous les achats, et cela nous donnera les métriques nécessaires. Nous pourrions obtenir certaines de ces valeurs en utilisant simplement les fonctions intégrées min() et max(), mais cela nécessiterait d'itérer tous les achats plus d'une fois, donc à la place, nous utilisons notre objet personnalisé, qui obtiendra ces valeurs en une seule itération.


Le code qui obtiendra les chiffres pour nous semble plutôt simple. C'est juste un objet avec une méthode qui traitera tous les prix en une seule fois, et, à chaque étape, mettra à jour la valeur de chaque métrique particulière qui nous intéresse. Tout d'abord, nous montrerons la première implémentation dans la liste suivante, et, plus loin dans ce guide (une fois que nous en saurons plus sur l'itération), nous revisiterons cette implémentation et en obtiendrons une version bien meilleure (et plus compacte). Pour l'instant, nous nous contentons de ce qui suit :


In [ ]:

class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self._initialize()

    def _initialize(self):
        try:
            first_value = next(self.purchases)
        except StopIteration:
            raise ValueError("no values provided")

        self.min_price = self.max_price = first_value
        self._update_avg(first_value)

    def process(self):
        for purchase_value in self.purchases:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self

    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value

    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value

    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases

    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1

    def __str__(self):
        return (
            f"{self.__class__.__name__}({self.min_price}, "
            f"{self.max_price}, {self.avg_price})"
        )


Cet objet recevra tous les totaux des achats et traitera les valeurs requises. Maintenant, nous avons besoin d'une fonction qui charge ces nombres dans quelque chose que cet objet peut traiter. Voici la première version :

In [ ]:
def _load_purchases(filename):
    purchases = []
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            purchases.append(float(price_raw))

    return purchases

Ce code fonctionne ; il charge tous les numéros du fichier dans une liste qui, une fois transmise à notre objet personnalisé, produira les numéros que nous voulons. Il a cependant un problème de performances. Si vous l'exécutez avec un ensemble de données assez volumineux, cela prendra un certain temps, et il peut même échouer si l'ensemble de données est suffisamment volumineux pour ne pas tenir dans la mémoire principale.

Si nous jetons un coup d'œil à notre code qui consomme ces données, il traite les achats, un à la fois, alors on peut se demander pourquoi notre producteur met tout en mémoire à la fois. Il crée une liste où il met tout le contenu du fichier, mais nous savons que nous pouvons faire mieux.

La solution est de créer un générateur. Au lieu de charger tout le contenu du fichier dans une liste, nous produirons les résultats un par un. Le code ressemblera maintenant à ceci :

In [ ]:
def load_purchases(filename):
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            yield float(price_raw)

Si vous mesurez le processus cette fois, vous remarquerez que l'utilisation de la mémoire a considérablement diminué. Nous pouvons également voir à quel point le code semble plus simple : il n'est pas nécessaire de définir la liste (donc, il n'est pas nécessaire de l'ajouter), et l'instruction return a également disparu.

Dans ce cas, la fonction load_purchases est une fonction génératrice, ou simplement un générateur. En Python, la simple présence du mot clé yield dans n'importe quelle fonction en fait un générateur, et, par conséquent, lors de son appel, rien d'autre que de créer un instance du générateur se produira


      >>> load_purchases("file")<
      generator object load_purchases at 0x...>


Un objet générateur est un itérable (nous reviendrons sur les itérables plus en détail plus loin), ce qui signifie qu'il peut fonctionner avec des boucles for. Notez que nous n'avons rien eu à changer sur le code consommateur - notre processeur de statistiques est resté le même, avec la boucle for non modifiée, après la nouvelle implémentation.

Travailler avec des itérables nous permet de créer ce genre d'abstractions puissantes qui sont polymorphes par rapport aux boucles for. Tant que nous conservons l'interface itérable, nous pouvons itérer sur cet objet de manière transparente.

Ce que nous explorons dans ce guide est un autre cas de code idiomatique qui se marie bien avec Python lui-même. Dans les guides précédents, nous avons vu comment nous pouvons implémenter nos propres gestionnaires de contexte pour connecter nos objets aux instructions with, ou comment pouvons-nous créer des objets conteneurs personnalisés pour tirer parti de l'opérateur in, ou des booléens pour l'instruction if, et ainsi de suite. C'est maintenant au tour de l'opérateur for, et pour cela, nous allons créer des itérateurs.

Avant d'entrer dans les détails et les nuances des générateurs, nous pouvons jeter un coup d'œil rapide sur la façon dont les générateurs se rapportent à un concept que nous avons déjà vu : les compréhensions. Un générateur sous la forme d'une compréhension est appelé une expression génératrice, et nous en parlerons brièvement dans la section suivante.

## Générateur d'expressions

Les générateurs économisent beaucoup de mémoire et, comme ce sont des itérateurs, ils constituent une alternative pratique aux autres itérables ou conteneurs qui nécessitent plus d'espace en mémoire, tels que les listes, les tuples ou les ensembles.

Tout comme ces structures de données, elles peuvent également être définies par compréhension, seulement qu'elles sont appelées une expression génératrice (il y a un débat en cours pour savoir si elles doivent être appelées compréhensions génératrices. Dans ce guide, nous nous référerons simplement à elles par leur nom canonique, mais n'hésitez pas à utiliser celui que vous préférez).

De la même manière, nous définirions une compréhension de liste. Si nous remplaçons les crochets par des parenthèses, nous obtenons un générateur qui résulte de l'expression. Les expressions de générateur peuvent également être transmises directement aux fonctions qui fonctionnent avec des itérables, telles que sum() et max() :

In [ ]:
[x**2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [ ]:
(x**2 for x in range(10))

<generator object <genexpr> at 0x7f1543dedad0>

In [ ]:
sum(x**2 for x in range(10))

285

    
    Transmettez toujours une expression de générateur, au lieu d'une
    compréhension de liste, aux fonctions qui attendent des itérables, telles
    que min(), max() et sum(). C'est plus efficace et Pythonic


Ce que signifie la recommandation précédente, c'est d'essayer d'éviter de passer des listes à des fonctions qui fonctionnent déjà avec des générateurs. L'exemple dans le code suivant est quelque chose que vous voudriez éviter et privilégier l'approche de la liste précédente :

In [ ]:
sum([x**2 for x in range(10)])  # here the list can be avoided

285

Et, bien sûr, vous pouvez affecter une expression génératrice à une variable et l'utiliser ailleurs (comme pour les compréhensions). Gardez à l'esprit qu'il y a une distinction importante dans ce cas, car nous parlons ici de générateurs. Une liste peut être réutilisée et itérée plusieurs fois, mais un générateur sera épuisé après avoir été itéré. Pour cette raison, assurez-vous que le résultat de l'expression n'est consommé qu'une seule fois, sinon vous obtiendrez des résultats inattendus.

Une approche courante consiste à créer de nouvelles expressions génératrices dans le code. De cette façon, le premier sera épuisé après avoir été itéré, mais un nouveau sera ensuite créé. Le chaînage des expressions du générateur de cette manière est utile et permet d'économiser de la mémoire et de rendre le code plus expressif, car il résout différentes itérations en différentes étapes. Un scénario où cela est utile est lorsque vous devez appliquer plusieurs filtres sur un itérable ; vous pouvez y parvenir en utilisant plusieurs expressions génératrices qui agissent comme des filtres chaînés. Maintenant que nous avons un nouvel outil dans notre boîte à outils (itérateurs), voyons comment nous pouvons l'utiliser pour écrire plus de code idiomatique

## Itérer idiomatiquement

Dans cette section, nous allons d'abord explorer quelques idiomes qui s'avèrent utiles lorsque nous devons gérer l'itération en Python. Ces recettes de code nous aideront à avoir une meilleure idée des types de choses que nous pouvons faire avec les générateurs (surtout après avoir déjà vu les expressions de générateur) et comment résoudre les problèmes typiques les concernant.

Une fois que nous aurons vu quelques idiomes, nous passerons à l'exploration plus approfondie de l'itération en Python, en analysant les méthodes qui rendent l'itération possible et le fonctionnement des objets itérables.

## Idiomes pour l'itération

Nous connaissons déjà la fonction intégrée enumerate() qui, étant donné un itérable, en renverra un autre sur lequel l'élément est un tuple, dont le premier élément est l'indice du second (correspondant à l'élément dans l'original itérable) :

In [ ]:
list(enumerate("abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

Nous souhaitons créer un objet similaire, mais à un niveau plus bas ; celui qui peut simplement créer une séquence infinie. Nous voulons un objet qui puisse produire une séquence de nombres, à partir d'un nombre de départ, sans aucune limite.


Un objet aussi simple que le suivant peut faire l'affaire. Chaque fois que nous appelons cet objet, nous obtenons le numéro suivant de la séquence à l'infini :

In [ ]:
class NumberSequence:
    """
    >>> seq = NumberSequence()
    >>> seq.next()
    0
    >>> seq.next()
    1
    >>> seq2 = NumberSequence(10)
    >>> seq2.next()
    10
    >>> seq2.next()
    11
    """

    def __init__(self, start=0):
        self.current = start

    def next(self):
        current = self.current
        self.current += 1
        return current

Sur la base de cette interface, nous devrions utiliser cet objet en appelant explicitement sa méthode next() :

In [ ]:
seq = NumberSequence()
seq.next()

0

In [ ]:
seq.next

1

In [ ]:
seq2 = NumberSequence(10)
seq2.next()

10

Mais avec ce code, nous ne pouvons pas reconstruire la fonction enumerate() comme nous le souhaiterions, car son interface ne prend pas en charge l'itération sur une boucle Python for régulière, ce qui signifie également que nous ne pouvons pas la transmettre en tant que paramètre aux fonctions qui attendent quelque chose à répéter. Remarquez comment le code suivant échoue :

In [ ]:
list(zip(NumberSequence(), "abcdef"))

TypeError: ignored

Le problème réside dans le fait que NumberSequence ne prend pas en charge l'itération. Pour résoudre ce problème, nous devons faire de l'objet un itérable en implémentant la méthode magique __iter__(). Nous avons également modifié la méthode next() précédente, en utilisant la méthode magique __next__, qui fait de l'objet un itérateur :

In [ ]:
class SequenceOfNumbers:
    """
    >>> list(zip(SequenceOfNumbers(), "abcdef"))
    [(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]
    >>> seq = SequenceOfNumbers(100)
    >>> next(seq)
    100
    >>> next(seq)
    101
    """

    def __init__(self, start=0):
        self.current = start

    def __next__(self):
        current = self.current
        self.current += 1
        return current

    def __iter__(self):
        return self

Cela présente un avantage : non seulement nous pouvons itérer sur l'élément, mais nous n'avons même plus besoin de la méthode .next() car avoir __next__() nous permet d'utiliser la fonction intégrée next() :

In [ ]:
list(zip(SequenceOfNumbers(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

In [ ]:
seq = SequenceOfNumbers(100)
next(seq)

100

Cela utilise le protocole d'itération. Semblable au protocole de gestionnaire de contexte que nous avons exploré dans les guide précédents, qui se compose des méthodes __enter__ et __exit__, ce protocole repose sur les méthodes __iter__ et __next__.

Avoir ces protocoles en Python a un avantage : tous ceux qui connaissent Python connaissent déjà cette interface, il y a donc une sorte de "contrat standard". Cela signifie, au lieu d'avoir à définir vos propres méthodes et à vous mettre d'accord avec l'équipe (ou tout lecteur potentiel du code), qu'il s'agit de la norme ou du protocole attendu avec lequel votre code fonctionne (comme avec notre méthode next() personnalisée dans le premier exemple) ; Python fournit déjà une interface et possède déjà un protocole. Nous n'avons qu'à le mettre en œuvre correctement

## La fonction next()

La fonction intégrée next() fera avancer l'itérable à son élément suivant et le renverra :

In [ ]:
word = iter("hello")
next(word)

'h'

In [ ]:
next(word)

'e'

In [ ]:
next(word)

'l'

In [ ]:
next(word)

'l'

In [ ]:
next(word)

'o'

Si l'itérateur n'a plus d'éléments à produire, l'exception StopIteration est levée :

In [ ]:
next(word)

StopIteration: ignored

Cette exception signale que l'itération est terminée et qu'il n'y a plus d'éléments à consommer. Si nous souhaitons gérer ce cas, en plus d'attraper l'exception StopIteration, nous pourrions fournir à cette fonction une valeur par défaut dans son deuxième paramètre. Si cela est fourni, ce sera la valeur de retour au lieu de lancer StopIteration :

In [ ]:
next(word, "default value")

'default value'

Il est conseillé d'utiliser la valeur par défaut la plupart du temps, pour éviter d'avoir des exceptions à l'exécution dans nos programmes. Si nous sommes absolument sûrs que l'itérateur auquel nous avons affaire ne peut pas être vide, il est toujours préférable d'être implicite (et intentionnel) à ce sujet, et de ne pas compter sur les effets secondaires des fonctions intégrées (c'est-à-dire pour affirmer correctement le cas ).

La fonction next() peut être très utile en combinaison avec des expressions génératrices, dans les situations où nous voulons rechercher les premiers éléments d'un itérable qui répond à certains critères. Nous verrons des exemples de cet idiome tout au long du guide, mais l'idée principale est d'utiliser cette fonction au lieu de créer une liste de compréhension puis de prendre son premier élément

## Utilisation d'un générateur

Le code précédent peut être simplifié de manière signicative en utilisant simplement un générateur. Les objets générateurs sont des itérateurs. De cette façon, au lieu de créer une classe, nous pouvons définir une fonction qui renvoie les valeurs nécessaires :

In [ ]:
def sequence(start=0):
  while True:        
    yield start        
    start += 1

Rappelez-vous que d'après notre première dénition, le mot-clé yield dans le corps de la fonction en fait un générateur. Puisqu'il s'agit d'un générateur, il est parfaitement judicieux de créer une boucle infinie comme celle-ci, car, lorsque cette fonction génératrice est appelée, elle exécutera tout le code jusqu'à ce que la prochaine instruction yield soit atteinte. Il produira sa valeur et y suspendra :

In [ ]:
seq = sequence(10)
next(seq)

10

In [ ]:
list(zip(sequence(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

Cette différence peut être considérée comme une analogie des différentes manières de créer un décorateur, comme nous l'avons exploré dans le guide précédent (avec un objet de fonctions). Ici aussi, on peut utiliser une fonction génératrice, ou un objet itérable, comme dans la section précédente. Dans la mesure du possible, la construction d'un générateur est recommandée, car elle est syntaxiquement plus simple, et donc plus facile à comprendre.

## Itertools

Travailler avec des objets itérables a l'avantage que le code se mélange mieux avec Python lui-même car l'itération est un composant clé du langage. En plus de cela, nous pouvons profiter pleinement du module itertools (ITER-01). En fait, le générateur sequence() que nous venons de créer est assez similaire à itertools.count(). Cependant, nous pouvons faire plus.

L'un des aspects les plus intéressants des itérateurs, des générateurs et des itertools est qu'il s'agit d'objets composables qui peuvent être enchaînés.

Par exemple, pour revenir à notre premier exemple qui traitait les achats afin d'obtenir des métriques, et si nous voulions faire la même chose, mais uniquement pour les valeurs dépassant un certain seuil ? L'approche naïve pour résoudre ce problème serait de placer la condition en itérant :

In [ ]:
def process(self):        
  for purchase in self.purchases:
    if purchase > 1000.0:                
      ...

Ce n'est pas seulement non Python, mais c'est aussi rigide (et la rigidité est un trait qui dénote un mauvais code). Il ne gère pas très bien les changements. Et si le nombre change maintenant ? Le passe-t-on par paramètre ? Et si nous en avons besoin de plus d'un ? Que faire si la condition est différente (inférieure à, par exemple) ? Passons-nous un lambda?

Ces questions ne doivent pas être répondues par cet objet, dont la seule responsabilité est de calculer un ensemble de métriques bien définies sur un flux d'achats représentés sous forme de nombres. Et, bien sûr, la réponse est non. Ce serait une énorme erreur de faire un tel changement (encore une fois, le code propre est flexible, et nous ne voulons pas le rendre rigide en couplant cet objet à des facteurs externes). Ces exigences devront être traitées ailleurs.

Il est préférable de garder cet objet indépendant de ses clients. Moins cette classe a de responsabilités, plus elle sera utile pour plus de clients, augmentant ainsi ses chances d'être réutilisée

Au lieu de changer ce code, nous allons le garder tel quel et supposer que les nouvelles données sont filtrées en fonction des exigences de chaque client de la classe.

Par exemple, si nous voulions traiter uniquement les 10 premiers achats qui s'élèvent à plus de 1000, nous procéderions comme suit :

In [ ]:
from itertools import islice

purchases = islice(filter(lambda p: p > 1000.0, purchases), 10)

stats = PurchasesStats(purchases).process()  

Il n'y a pas de pénalisation de la mémoire pour ce filtrage car comme ce sont tous des générateurs, l'évaluation est toujours paresseuse. Cela nous donne le pouvoir de penser comme si nous avions filtré l'ensemble d'un seul coup puis passé à l'objet, mais sans vraiment tout mettre en mémoire. entre la mémoire et l'utilisation du processeur. Bien que le code puisse utiliser moins de mémoire, cela pourrait prendre plus de temps CPU, mais la plupart du temps, cela est acceptable, lorsque nous devons traiter de nombreux objets en mémoire tout en gardant le code maintenable

## Simplifier le code grâce aux itérateurs

Maintenant, nous allons brièvement discuter de certaines situations qui peuvent être améliorées à l'aide d'itérateurs, et occasionnellement du module itertools. Après avoir discuté de chaque cas, et de son optimisation proposée, nous clôturerons chaque point par un corollaire

## Itérations répétées

Maintenant que nous en avons vu plus sur les itérateurs et introduit le module itertools, nous pouvons vous montrer comment l'un des premiers exemples de ce 
guide (celui pour le calcul de statistiques sur certains achats) peut être considérablement simplifié :

In [ ]:
from itertools import tee

def process_purchases(purchases):
    min_, max_, avg = tee(purchases, 3)
    return min(min_), max(max_), median(avg)

Dans cet exemple, itertools.tee divisera l'itérable d'origine en trois nouveaux. Nous utiliserons chacun d'eux pour les différents types d'itérations dont nous avons besoin, sans avoir besoin de répéter trois boucles différentes sur les achats.

Le lecteur peut simplement vérifier que si l'on passe un objet itérable en paramètre achats, celui-ci n'est parcouru qu'une seule fois (grâce à la fonction itertools.tee [TEE]), ce qui était notre principale exigence. Il est également possible de vérifier en quoi cette version est équivalente à notre implémentation d'origine. Dans ce cas, il n'est pas nécessaire de lever manuellement ValueError car le passage d'une séquence vide à la fonction min() le fera.


Le module itertools contient de nombreuses fonctions utiles et de belles abstractions qui s'avèrent utiles lorsqu'il s'agit d'itérations en Python. Il contient également de bonnes recettes sur la façon de résoudre des problèmes d'itération typiques de manière idiomatique. En guise de conseil général, si vous réfléchissez à la façon de résoudre un problème particulier impliquant une itération, jetez un œil à ce module. Même si la réponse n'est pas littéralement là, ce sera une bonne inspiration.

## Boucles imbriquées

Dans certaines situations, nous devons itérer sur plus d'une dimension, à la recherche d'une valeur, et les boucles imbriquées sont la première idée. Lorsque la valeur est trouvée, nous devons arrêter l'itération, mais le mot-clé break ne fonctionne pas entièrement car nous devons nous échapper de deux (ou plus) boucles for, pas seulement une.

Quelle serait la solution à cela ? Un drapeau signalant l'évasion ? Non. Lever une exception ? Non, ce serait la même chose que le drapeau, mais encore pire parce que nous savons que les exceptions ne doivent pas être utilisées pour la logique de flux de contrôle. Déplacer le code vers une fonction plus petite et la renvoyer ? Fermer, mais pas tout à fait.

La réponse est, dans la mesure du possible, d'aplatir l'itération en une seule boucle for. C'est le genre de code que nous aimerions éviter :

In [ ]:
def search_nested_bad(array, desired_value):
    """Example of an iteration in a nested loop."""
    coords = None
    for i, row in enumerate(array):
        for j, cell in enumerate(row):
            if cell == desired_value:
                coords = (i, j)
                break

        if coords is not None:
            break

    if coords is None:
        raise ValueError(f"{desired_value} not found")

    print("value %r found at [%i, %i]", desired_value, *coords)
    return coords

Et voici une version simplifiée de celui-ci qui ne repose pas sur des drapeaux pour signaler la terminaison et qui a une structure d'itération plus simple et plus compacte :

In [ ]:
def _iterate_array2d(array2d):
    for i, row in enumerate(array2d):
        for j, cell in enumerate(row):
            yield (i, j), cell


def search_nested(array, desired_value):
    """"Searching in multiple dimensions with a single loop."""
    try:
        coord = next(
            coord
            for (coord, cell) in _iterate_array2d(array)
            if cell == desired_value
        )
    except StopIteration as e:
        raise ValueError(f"{desired_value} not found") from e

    print("value %r found at [%i, %i]", desired_value, *coord)
    return coord

Il convient de mentionner comment le générateur auxiliaire qui a été créé fonctionne comme une abstraction pour l'itération requise. Dans ce cas, nous avons juste besoin d'itérer sur deux dimensions, mais si nous en avions besoin de plus, un objet différent pourrait gérer cela sans que le client ait besoin de le savoir. C'est l'essence du modèle de conception de l'itérateur, qui, en Python, est transparent, car il prend en charge automatiquement les objets itérateurs, ce qui est le sujet traité dans la section suivante.

    Essayez de simplifier l'itération autant que possible avec autant
    d'abstractions que nécessaire, en aplatissant les boucles dans la mesure du
     possible.

Espérons que cet exemple vous inspirera pour vous faire une idée que nous pouvons utiliser des générateurs pour quelque chose de plus que simplement économiser de la mémoire. Nous pouvons tirer parti de l'itération comme une abstraction. C'est-à-dire que nous pouvons créer des abstractions non seulement en définissant des classes ou des fonctions, mais aussi en tirant parti de la syntaxe de Python. De la même manière que nous avons vu comment faire abstraction d'une certaine logique derrière un gestionnaire de contexte (donc nous ne connaissons pas les détails de ce qui se passe sous l'instruction with), nous pouvons faire de même avec les itérateurs (ainsi nous pouvons oublier le sous-jacent logique d'une boucle for)

C'est pourquoi nous allons commencer à explorer le fonctionnement du modèle d'itérateur en Python, en commençant par la section suivante

## Le modèle d'itérateur en Python
Ici, nous ferons un petit détour par les générateurs pour comprendre plus en profondeur l'itération en Python. Les générateurs sont un cas particulier d'objets itérables, mais l'itération en Python va au-delà des générateurs, et être capable de créer de bons objets itérables nous donnera la possibilité de créer un code plus efficace, compact et lisible.

Dans les listes de code précédentes, nous avons vu des exemples d'objets itérables qui sont également des itérateurs, car ils implémentent à la fois les méthodes magiques __iter__() et __next__(). Bien que ce soit ne en général, il n'est pas strictement nécessaire qu'ils implémentent toujours les deux méthodes, et nous montrerons ici les différences subtiles entre un objet itérable (celui qui implémente __iter__) et un itérateur (qui implémente __next__)

Nous explorons également d'autres sujets liés aux itérations, tels que les séquences et les objets conteneurs.

## L'interface pour l'itération

Un itérable est un objet qui prend en charge l'itération, ce qui, à un niveau très élevé, signifie que nous pouvons exécuter une boucle for .. in ... dessus, et cela fonctionnera sans aucun problème. Cependant, iterable ne signifie pas la même chose que iterator.

De manière générale, un itérable est simplement quelque chose que nous pouvons itérer, et il utilise un itérateur pour le faire. Cela signifie que dans la méthode magique __iter__, nous aimerions retourner un itérateur, à savoir un objet avec une méthode __next__() implémentée

Un itérateur est un objet qui ne sait que produire une série de valeurs, une à la fois, lorsqu'il est appelé par la fonction intégrée next() déjà explorée, alors que l'itérateur n'est pas appelé, il est simplement figé, inactif par jusqu'à ce qu'il soit à nouveau appelé pour la prochaine valeur à produire. En ce sens, les générateurs sont des itérateurs.

|  Python concept | Magic method  |   Consideration |
|:-:|---|---|
|   Iterable|  __iter__ | ils travaillent avec un itérateur pour construire la logique d'itération. Ces objets peuvent être itérés dans un for ... in ... : boucle  |
|  Iterator |__next__   |  Définissez la logique de production des valeurs une par une. L'exception StopIteration signale que l'itération est terminée. Les valeurs peuvent être obtenues une par une via la fonction intégrée next(). |



Dans le code suivant, nous verrons un exemple d'objet itérateur qui n'est pas itérable - il ne prend en charge que l'appel de ses valeurs, une à la fois. Ici, la séquence de noms fait simplement référence à une série de nombres consécutifs, pas au concept de séquence en Python, que nous explorerons plus tard


In [ ]:
class SequenceIterator:
    """
    >>> si = SequenceIterator(1, 2)
    >>> next(si)
    1
    >>> next(si)
    3
    >>> next(si)
    5
    """

    def __init__(self, start=0, step=1):
        self.current = start
        self.step = step

    def __next__(self):
        value = self.current
        self.current += self.step
        return value

Notez que nous pouvons obtenir les valeurs de la séquence une par une, mais nous ne pouvons pas itérer sur cet objet (c'est une chance car cela entraînerait sinon une boucle sans fin):

In [ ]:
si = SequenceIterator(1, 2)
next(si)

1

In [ ]:
for _ in SequenceIterator(): pass

TypeError: ignored

Le message d'erreur est clair, car l'objet n'implémente pas __iter__().Juste à des fins explicatives, nous pouvons séparer l'itération dans un autre objet (encore une fois, il suffirait de faire en sorte que l'objet implémente à la fois __iter__ et __next__, mais en le faisant séparément aidera à clarifier le point distinctif que nous essayons de faire dans cette explication)

## Objets de séquence en tant qu'itérables

Comme nous venons de le voir, si un objet implémente la méthode magique __iter__(), cela signifie qu'il peut être utilisé dans une boucle for. Bien qu'il s'agisse d'une fonctionnalité intéressante, ce n'est pas la seule forme d'itération possible que nous puissions réaliser. Lorsque nous écrivons une boucle for, Python essaiera de voir si l'objet que nous utilisons implémente __iter__, et si c'est le cas, il l'utilisera pour construire l'itération, mais si ce n'est pas le cas, il existe des options de secours.

Si l'objet se trouve être une séquence (c'est-à-dire qu'il implémente les méthodes magiques __getitem__() et __len__()), il peut également être itéré. Si tel est le cas, l'interpréteur fournira alors des valeurs en séquence, jusqu'à ce que l'exception IndexError soit levée, ce qui, de manière analogue à la StopIteration susmentionnée, signale également l'arrêt de l'itération.

Dans le seul but d'illustrer un tel comportement, nous allons exécuter l'expérience suivante qui montre un objet séquence qui implémente map() sur une plage de nombres :

In [ ]:
import logging

logger = logging.getLogger(__name__)

class MappedRange:
    """Apply a transformation to a range of numbers."""

    def __init__(self, transformation, start, end):
        self._transformation = transformation
        self._wrapped = range(start, end)

    def __getitem__(self, index):
        value = self._wrapped.__getitem__(index)
        result = self._transformation(value)
        logger.info("Index %d: %s", index, result)
        return result

    def __len__(self):
        return len(self._wrapped)

Gardez à l'esprit que cet exemple est uniquement conçu pour illustrer qu'un objet tel que celui-ci peut être itéré avec une boucle for régulière. Il y a une ligne de journalisation placée dans la méthode __getitem__ pour explorer quelles valeurs sont transmises pendant l'itération de l'objet, comme nous pouvons le voir dans le test suivant :

In [ ]:
mr = MappedRange(abs, -10, 5)
mr[0]

10

In [ ]:
mr[-1]

4

In [ ]:
list(mr)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4]

En guise de mise en garde, il est important de souligner que s'il est utile de le savoir, c'est aussi un mécanisme de secours lorsque l'objet n'implémente pas __iter__, donc la plupart du temps nous voudrons recourir à ces méthodes en pensant sur la création de séquences appropriées, et pas seulement sur les objets que nous voulons parcourir


    Lorsque vous envisagez de concevoir un objet pour l'itération, privilégiez
    un objet itérable approprié (avec __iter__), plutôt qu'une séquence qui
    peut aussi être itérée par coïncidence
  

Les itérables sont une partie importante de Python, non seulement en raison des capacités qu'ils nous offrent en tant qu'ingénieurs logiciels, mais aussi parce qu'ils jouent un rôle fondamental dans les composants internes de Python.

Nous avons vu dans Une brève introduction au code asynchrone au guide 2, Pythonic Code, comment lire du code asynchrone. Maintenant que nous avons également exploré les itérateurs en Python, nous pouvons voir comment ces deux concepts sont liés. En particulier, la section suivante explore les coroutines, et nous verrons comment les itérateurs sont au cœur de celles-ci

## Coroutines

L'idée d'une coroutine est d'avoir une fonction, dont l'exécution peut être suspendue à un moment donné, pour être reprise ultérieurement. En ayant ce genre de fonctionnalité, un programme pourrait être capable de suspendre une partie du code, afin d'envoyer quelque chose d'autre pour le traitement, puis de revenir à ce point d'origine pour reprendre.


Comme nous le savons déjà, les objets générateurs sont itérables. Ils implémentent __iter__() et __next__(). Ceci est fourni automatiquement par Python afin que lorsque nous créons une fonction d'objet générateur, nous obtenions un objet qui peut être itéré ou avancé via la fonction next().

Outre cette fonctionnalité de base, ils ont plus de méthodes pour pouvoir fonctionner comme des coroutines (PEP-342). Ici, nous explorerons comment les générateurs ont évolué en coroutines pour prendre en charge la base de la programmation asynchrone avant d'entrer plus en détail dans la section suivante, où nous explorerons les nouvelles fonctionnalités de Python et la syntaxe qui couvre la programmation asynchrone.

Les méthodes de base ajoutées dans PEP-342 pour prendre en charge les coroutines sont les suivantes :


* .close()
*  .throw(ex_type[, ex_value[, ex_traceback]])
*  .send(value)


Python tire parti des générateurs pour créer des coroutines. Parce que les générateurs peuvent naturellement se suspendre, ils constituent un point de départ pratique. Mais les générateurs n'étaient pas suffisants comme on le pensait à l'origine, alors ces méthodes ont été ajoutées. En effet, il ne suffit généralement pas de pouvoir suspendre une partie du code ; vous voudriez également communiquer avec lui (transmettre des données et signaler des changements dans le contexte)

En explorant chaque méthode plus en détail, nous pourrons en savoir plus sur les éléments internes des coroutines en Python. Après cela, je présenterai une autre récapitulation du fonctionnement de la programmation asynchrone, mais contrairement à celle présentée au guide 2, Pythonic Code, celle-ci se rapportera aux concepts internes que nous venons d'apprendre.

## Les méthodes de l'interface du générateur

Dans cette section, nous allons explorer ce que fait chacune des méthodes susmentionnées, comment elles fonctionnent et comment elles devraient être utilisées. En comprenant comment utiliser ces méthodes, nous pourrons utiliser des coroutines simples. Plus tard, nous explorerons des utilisations plus avancées des coroutines, et comment déléguer à des sous-générateurs (coroutines) afin de refactoriser le code, et comment pour orchestrer différentes coroutines.



## close()

Lors de l'appel de cette méthode, le générateur recevra l'exception GeneratorExit. S'il n'est pas géré, alors le générateur se terminera sans produire plus de valeurs, et son itération s'arrêtera.

Cette exception peut être utilisée pour gérer un état de fin. En général, si notre coroutine effectue une sorte de gestion des ressources, nous voulons intercepter cette exception et utiliser ce bloc de contrôle pour libérer toutes les ressources détenues par la coroutine. Cela revient à utiliser un gestionnaire de contexte ou à placer le code dans le bloc finally d'un contrôle d'exception, mais la gestion de cette exception la rend plus explicite.


Dans l'exemple suivant, nous avons une coroutine qui utilise un objet gestionnaire de base de données qui détient une connexion à une base de données et exécute des requêtes dessus, diffusant des données par pages d'une longueur fixe (au lieu de lire tout ce qui est disponible à la fois ):

In [ ]:
import time


class DBHandler:
    """Simulate reading from the database by pages."""

    def __init__(self, db):
        self.db = db
        self.is_closed = False

    def read_n_records(self, limit):
        return [(i, f"row {i}") for i in range(limit)]

    def close(self):
        logger.debug("closing connection to database %r", self.db)
        self.is_closed = True

def stream_db_records(db_handler):
    """Example of .close()
    >>> streamer = stream_db_records(DBHandler("testdb"))  # doctest: +ELLIPSIS
    >>> len(next(streamer))
    10
    >>> len(next(streamer))
    10
    """
    try:
        while True:
            yield db_handler.read_n_records(10)
            time.sleep(0.1)
    except GeneratorExit:
        db_handler.close()


A chaque appel au générateur, il renverra 10 lignes obtenues à partir du gestionnaire de base de données, mais lorsque nous décidons de terminer explicitement l'itération et appelons close(), nous souhaitons également fermer la connexion à la base de données :

In [ ]:
streamer = stream_db_records(DBHandler("testdb"))
next(streamer)

[(0, 'row 0'),
 (1, 'row 1'),
 (2, 'row 2'),
 (3, 'row 3'),
 (4, 'row 4'),
 (5, 'row 5'),
 (6, 'row 6'),
 (7, 'row 7'),
 (8, 'row 8'),
 (9, 'row 9')]

In [ ]:
next(streamer)

[(0, 'row 0'),
 (1, 'row 1'),
 (2, 'row 2'),
 (3, 'row 3'),
 (4, 'row 4'),
 (5, 'row 5'),
 (6, 'row 6'),
 (7, 'row 7'),
 (8, 'row 8'),
 (9, 'row 9')]

In [ ]:
streamer.close()

closing connection to database %r testdb


Cette méthode est destinée à être utilisée pour le nettoyage des ressources, vous l'utiliserez donc généralement pour libérer manuellement des ressources lorsque vous ne pourriez pas le faire automatiquement (par exemple, si vous n'avez pas utilisé de gestionnaire de contexte). Ensuite, nous verrons comment passer des exceptions au générateur.

## throw(ex_type[, ex_value[, ex_traceback]])

Cette méthode lèvera l'exception à la ligne où le générateur est actuellement suspendu. Si le générateur gère l'exception qui a été envoyée, le code de cette clause except particulière sera appelé ; sinon, l'exception se propagera à l'appelant.

Ici, nous modifions légèrement l'exemple précédent pour montrer la différence lorsque nous utilisons cette méthode pour une exception gérée par la coroutine, et quand elle ne l'est pas :


In [ ]:
class CustomException(Exception):
    """An exception of the domain model."""


def stream_data(db_handler):
    """Test the ``.throw()`` method.
    >>> streamer = stream_data(DBHandler("testdb"))
    >>> len(next(streamer))
    10
    """
    while True:
        try:
            yield db_handler.read_n_records(10)
        except CustomException as e:
            print("controlled error %r, continuing", e)
        except Exception as e:
            print("unhandled error %r, stopping", e)
            db_handler.close()
            break

Maintenant, cela fait partie du flux de contrôle de recevoir une CustomException, et, dans un tel cas, le générateur enregistrera un message informatif (bien sûr, nous pouvons l'adapter en fonction de notre logique métier sur chaque cas), et déplacer à la prochaine déclaration de yield, qui est la ligne où la coroutine lit à partir de la base de données et renvoie ces données.

Cet exemple particulier gère toutes les exceptions, mais si le dernier bloc (sauf Exception :) n'était pas là, le résultat serait que le générateur est levé à la ligne où le générateur est en pause (encore une fois, yield), et il se propagera à partir de là à l'appelant:

In [ ]:
streamer = stream_data(DBHandler("testdb"))
next(streamer)

[(0, 'row 0'),
 (1, 'row 1'),
 (2, 'row 2'),
 (3, 'row 3'),
 (4, 'row 4'),
 (5, 'row 5'),
 (6, 'row 6'),
 (7, 'row 7'),
 (8, 'row 8'),
 (9, 'row 9')]

In [ ]:
streamer.throw(CustomException)

controlled error %r, continuing 


[(0, 'row 0'),
 (1, 'row 1'),
 (2, 'row 2'),
 (3, 'row 3'),
 (4, 'row 4'),
 (5, 'row 5'),
 (6, 'row 6'),
 (7, 'row 7'),
 (8, 'row 8'),
 (9, 'row 9')]

In [ ]:
streamer.throw(RuntimeError)

unhandled error %r, stopping 
closing connection to database %r testdb


StopIteration: ignored

Lorsque notre exception du domaine a été reçue, le générateur a continué. Cependant, lorsqu'il a reçu une autre exception qui n'était pas attendue, le bloc par défaut a détecté l'endroit où nous avons fermé la connexion à la base de données et terminé l'itération, ce qui a entraîné l'arrêt du générateur. Comme nous pouvons le voir dans le StopIteration qui a été déclenché, ce générateur ne peut pas être itéré davantage.

# send(value)

Dans l'exemple précédent, nous avons créé un générateur simple qui lit les lignes d'une base de données, et lorsque nous avons souhaité terminer son itération, ce générateur a libéré les ressources liées à la base de données. C'est un bon exemple d'utilisation de l'une des méthodes fournies par les générateurs (close()), mais nous pouvons faire plus. Une observation du générateur est qu'il lisait un nombre fixe de lignes de la base de données.

Une observation du générateur est qu'il lisait un nombre fixe de lignes de la base de données. Nous aimerions paramétrer ce nombre (10) afin de pouvoir le changer au cours des différents appels. Malheureusement, la fonction next() ne nous fournit pas d'options pour cela. Mais heureusement, nous avons send()

In [ ]:
def _stream_db_records(db_handler):
    retrieved_data = None
    previous_page_size = 10
    try:
        while True:
            page_size = yield retrieved_data
            if page_size is None:
                page_size = previous_page_size

            previous_page_size = page_size

            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

L'idée est que nous avons maintenant rendu la coroutine capable de recevoir des valeurs de l'appelant au moyen de la méthode send(). Cette méthode est celle qui distingue réellement un générateur d'une coroutine car lorsqu'elle est utilisée, cela signifie que le mot-clé yield apparaîtra à droite de l'instruction et que sa valeur de retour sera affectée à autre chose.

Dans les coroutines, on trouve généralement le mot-clé yield à utiliser sous la forme suivante :

    receive = yield produced



yield, dans ce cas, fera deux choses. Il renverra le produit à l'appelant, qui le récupérera au prochain tour d'itération (après avoir appelé next(), par exemple), et il y sera suspendu. Plus tard, l'appelant voudra renvoyer une valeur à la coroutine en utilisant la méthode send(). Cette valeur deviendra le résultat de l'instruction yield, affectée dans ce cas à la variable nommée receive.


L'envoi de valeurs à la coroutine ne fonctionne que lorsque celle-ci est suspendue à une instruction yield, en attendant que quelque chose se produise. Pour que cela se produise, la coroutine devra être avancée à ce statut. La seule façon de le faire est d'appeler next() dessus. Cela signifie qu'avant d'envoyer quoi que ce soit à la coroutine, cela doit être avancé au moins une fois via la méthode next(). Ne pas le faire entraînera une exception :

In [ ]:
def coro():
  y = yield
  
c = coro()
c.send(1)

TypeError: ignored

    N'oubliez jamais de faire avancer une coroutine en appelant next() avant de lui envoyer des valeurs


Revenons à notre exemple. Nous modifions la façon dont les éléments sont produits ou diffusés pour lui permettre de recevoir la longueur des enregistrements qu'il s'attend à lire à partir de la base de données.


La première fois que nous appelons next(), le générateur avance jusqu'à la ligne contenant le yield ; il fournira une valeur à l'appelant (Aucun, comme défini dans la variable), et il y sera suspendu). A partir de là, nous avons deux options. Si nous choisissons de faire avancer le générateur en appelant next(), la valeur par défaut de 10 sera utilisée et il continuera comme d'habitude. En effet, appeler next() est techniquement identique à send(None), mais cela est couvert dans l'instruction if qui gérera la valeur que nous avons précédemment définie.

Si par contre on décide de fournir une valeur explicite via send(<valeur>), celle-ci deviendra le résultat de l'instruction yield, qui sera affectée à la variable contenant la longueur de la page à utiliser, qui , à son tour, sera utilisé pour lire à partir de la base de données.

Les appels successifs auront cette logique, mais le point important est que maintenant nous pouvons changer dynamiquement la longueur des données à lire au milieu de l'itération, à tout moment.

Maintenant que nous comprenons comment fonctionne le code précédent, la plupart des Pythonistes s'attendraient à une version simplifiée de celui-ci (après tout, Python est aussi une question de brièveté et de code propre et compact) :

In [ ]:
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

Cette version est non seulement plus compacte, mais elle illustre aussi mieux l'idée. Les parenthèses autour de yield indiquent plus clairement qu'il s'agit d'une instruction (pensez-y comme s'il s'agissait d'un appel de fonction) et que nous utilisons le résultat pour le comparer à la valeur précédente.

Cela fonctionne comme prévu, mais nous devons toujours nous rappeler de faire avancer la coroutine avant de lui envoyer des données. Si nous oublions d'appeler le premier next(), nous obtiendrons une TypeError. Cet appel pourrait être ignoré pour nos besoins car il ne renvoie rien que nous utiliserons.

Ce serait bien si nous pouvions utiliser la coroutine directement, juste après sa création, sans avoir à nous rappeler d'appeler next() la première fois, à chaque fois que nous allons l'utiliser. Certains auteurs (PYCOOK) ont imaginé un décorateur intéressant pour y parvenir. L'idée de ce décorateur est de faire avancer la coroutine, donc la définition suivante fonctionne automatiquement

In [ ]:
def prepare_coroutine(coroutine):
    def wrapped(*args, **kwargs):
        advanced_coroutine = coroutine(*args, **kwargs)
        next(advanced_coroutine)
        return advanced_coroutine

    return wrapped


@prepare_coroutine
def auto_stream_db_records(db_handler):
    """This coroutine is automatically advanced so it doesn't need the first
    next() call.
    """
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

In [ ]:
streamer = auto_stream_db_records(DBHandler("testdb"))
len(streamer.send(5))

5

closing connection to database %r testdb


Gardez à l'esprit que ce sont les principes fondamentaux du fonctionnement des coroutines en Python. En suivant ces exemples, vous aurez une idée de ce qui se passe réellement en Python lorsque vous travaillez avec des coroutines. Cependant, dans Python moderne, vous n'écririez généralement pas ces types de coroutines par vous-même, car une nouvelle syntaxe est disponible (que nous avons mentionnée, mais nous y reviendrons pour voir comment elles se rapportent aux idées que nous venons de voir)

Avant de sauter dans les nouvelles capacités syntaxiques, nous devons explorer le dernier saut que les coroutines ont fait en termes de fonctionnalités supplémentaires, afin de combler les lacunes manquantes. Après cela, nous pourrons comprendre la signification de chaque mot-clé et instruction utilisés dans la programmation asynchrone

## Coroutines plus avancées

Jusqu'à présent, nous avons une meilleure compréhension des coroutines et nous pouvons en créer de simples pour gérer de petites tâches. Nous pouvons dire que ces coroutines sont, en fait, juste des générateurs plus avancés (et ce serait vrai, les coroutines ne sont que des générateurs fantaisistes), mais, si nous voulons réellement commencer à prendre en charge des scénarios plus complexes, nous devons généralement opter pour une conception qui gère de nombreuses coroutines simultanément, et qui nécessite plus de fonctionnalités.

Lors de la manipulation de nombreuses coroutines, nous rencontrons de nouveaux problèmes. Au fur et à mesure que le flux de contrôle de notre application devient plus complexe, nous souhaitons transmettre des valeurs de haut en bas de la pile (ainsi que des exceptions), être en mesure de capturer des valeurs à partir de sous-coroutines que nous pourrions appeler à n'importe quel niveau, et enfin, programmer plusieurs coroutines pour courir vers un objectif commun.

Pour simplifier les choses, les générateurs ont dû être étendus une fois de plus. C'est ce que PEP-380 a résolu en modifiant la sémantique des générateurs afin qu'ils puissent renvoyer des valeurs et en introduisant le nouveau rendement de la construction

## Renvoyer des valeurs dans des coroutines

Comme présenté au début de ce chapitre, l'itération est un mécanisme qui appelle next() sur un objet itérable plusieurs fois jusqu'à ce qu'une exception StopIteration soit déclenchée.

Jusqu'à présent, nous avons exploré la nature itérative des générateurs - nous produisons des valeurs une à la fois et, en général, nous ne nous soucions que de chaque valeur telle qu'elle est produite à chaque étape de la boucle for. C'est une façon très logique de penser aux générateurs, mais les coroutines ont une idée différente ; même s'ils sont techniquement générateurs, ils n'ont pas été conçus dans l'idée d'itération, mais dans le but de suspendre l'exécution du code jusqu'à ce qu'il reprenne plus tard.

C'est un défi intéressant; lorsque nous concevons une coroutine, nous nous soucions généralement plus de suspendre l'état plutôt que d'itérer (et itérer une coroutine serait un cas étrange). Le défi réside dans le fait qu'il est facile de mélanger les deux. Ceci est dû à un détail technique de mise en œuvre ; le support des coroutines en Python a été construit sur des générateurs

Si nous voulons utiliser des coroutines pour traiter certaines informations et suspendre leur exécution, il serait logique de les considérer comme des threads légers (ou des threads verts, comme on les appelle sur d'autres plates-formes). Dans un tel cas, il serait logique qu'ils puissent renvoyer des valeurs, un peu comme appeler n'importe quelle autre fonction régulière.

Mais rappelons-nous que les générateurs ne sont pas des fonctions régulières, donc dans un générateur, la construction value = generator() ne fera rien d'autre que de créer un objet générateur. Quelle serait la sémantique pour qu'un générateur renvoie une valeur ? Il faudra que ce soit après l'itération

Lorsqu'un générateur renvoie une valeur, son itération est immédiatement arrêtée (il ne peut plus être itéré). Pour préserver la sémantique, l'exception StopIteration est toujours déclenchée et la valeur à renvoyer est stockée dans l'objet exception. C'est la responsabilité de l'appelant de l'attraper.

Dans l'exemple suivant, nous créons un générateur simple qui produit deux valeurs puis en renvoie une troisième. Remarquez comment nous devons attraper l'exception afin d'obtenir cette valeur, et comment elle est stockée précisément à l'intérieur de l'exception sous l'attribut nommé value :

In [ ]:
def generator():
  yield 1
  yield 2
  
  return 3
  
value = generator()
next(value)

1

In [ ]:
next(value)

2

In [ ]:
try:
  next(value)
except StopIteration as e:
  print(f">>>>>> returned value: {e.value}")

>>>>>> returned value: 3


Comme nous le verrons plus tard, ce mécanisme est utilisé pour que les coroutines renvoient des valeurs. Avant PEP-380, cela n'avait aucun sens, et toute tentative d'avoir une instruction return dans un générateur était considérée comme une erreur de syntaxe. Mais maintenant, l'idée est que, lorsque l'itération est terminée, on veut retourner une valeur finale, et le moyen de la fournir est de la stocker dans l'exception qui est levée à la fin de l'itération (StopIteration). Ce n'est peut-être pas l'approche la plus propre, mais c'est complètement rétrocompatible, car cela ne change pas l'interface du générateur

## Déléguer à des coroutines plus petites - la syntaxe "yield from" 

La fonctionnalité précédente est intéressante dans le sens où elle ouvre de nombreuses nouvelles possibilités avec les coroutines (générateurs), maintenant qu'elles peuvent renvoyer des valeurs. Mais cette fonctionnalité, en elle-même, ne serait pas aussi utile sans une prise en charge appropriée de la syntaxe, car attraper la valeur renvoyée de cette façon est un peu fastidieux.

C'est l'une des principales caractéristiques du rendement de la syntaxe yield from (que nous reviendrons en détail), il peut collecter la valeur renvoyée par un sous-générateur. N'oubliez pas que nous avons dit que renvoyer des données dans un générateur était bien, mais que, malheureusement, écrire des instructions sous la forme value = generator() ne fonctionnerait pas ? Eh bien, les écrire sous la forme value = yield from generator() serait

## l'utilisation la plus simple de yield from

Dans sa forme la plus élémentaire, la nouvelle syntaxe yield from peut être utilisée pour enchaîner des générateurs de boucles for imbriquées en une seule, qui se terminera par une seule chaîne de toutes les valeurs dans un flux continu

Un exemple canonique consiste à créer une fonction similaire à itertools.chain() à partir de la bibliothèque standard. C'est une fonction très intéressante car elle vous permet de passer n'importe quel nombre d'itérables et de les renvoyer tous ensemble dans un seul flux. L'implémentation naïve pourrait ressembler à ceci :

In [ ]:
def chain(*iterables):    
  for it in iterables:
    for value in it:            
      yield value

Il reçoit un nombre variable d'itérables, les traverse tous, et puisque chaque valeur est itérable, il prend en charge une construction for... in.., nous avons donc une autre boucle for pour obtenir chaque valeur à l'intérieur de chaque itérable particulier, qui est produit par la fonction appelante. 

Cela peut être utile dans plusieurs cas, comme enchaîner des générateurs ou essayer d'itérer des choses qu'il ne serait normalement pas possible de comparer en une seule fois (comme des listes avec des tuples, etc.). 

Cependant, le rendement de la syntaxe permet nous permet d'aller plus loin et d'éviter la boucle imbriquée car elle est capable de produire directement les valeurs d'un sous-générateur. Dans ce cas, nous pourrions simplifier le code comme ceci :

In [ ]:
def chain(*iterables):    
  for it in iterables:        
    yield from it

Notez que pour les deux implémentations, le comportement du générateur est exactement le même


Cela signifie que nous pouvons utiliser le rendement de n'importe quel autre itérable, et cela fonctionnera comme si le générateur de niveau supérieur (celui que le rendement utilise) générait lui-même ces valeurs. Cela fonctionne avec n'importe quel itérable, et même les expressions de générateur ne sont pas

 ce n'est pas l'exception. Maintenant que nous connaissons sa syntaxe, voyons comment nous pourrions écrire une fonction génératrice simple qui produira toutes les puissances d'un nombre (par exemple, si elle est fournie avec all_powers(2, 3), elle devra produire 2^ 0, 2^1,... 2^3) :

In [ ]:
def all_powers(n, pow):    
  yield from (n ** i for i in range(pow + 1))

In [ ]:
list(all_powers(2,3))

[1, 2, 4, 8]

Bien que cela simplifie un peu la syntaxe, enregistrer une ligne d'une instruction for n'est pas un gros avantage, et cela ne justifierait pas d'ajouter un tel changement au langage. En effet, ce n'est en fait qu'un effet secondaire et le vrai la raison d'être du rendement de la construction est ce que nous allons explorer dans les deux sections suivantes

## Capture de la valeur renvoyée par un sous-générateur 

Dans l'exemple suivant, nous avons un générateur qui appelle deux autres générateurs imbriqués, produisant des valeurs dans une séquence. Chacun de ces générateurs imbriqués renvoie une valeur, et nous verrons comment le générateur de niveau supérieur est capable de capturer efficacement la valeur de retour puisqu'il appelle les générateurs internes via yield from :

In [ ]:
def sequence(name, start, end):    
  print("%s started at %i", name, start)    
  yield from range(start, end)    
  print("%s finished at %i", name, end)    
  return end
  
def main():    
  step1 = yield from sequence("first", 0, 5)    
  step2 = yield from sequence("second", step1, 10)    
  return step1 + step2

In [ ]:
g = main()
next(g)

%s started at %i first 0


0

In [ ]:
next(g)

1

In [ ]:
next(g)

2

In [ ]:
next(g)

3

In [ ]:
next(g)

4

In [ ]:
next(g)

%s finished at %i first 5
%s started at %i second 5


5

In [ ]:
next(g)

%s finished at %i second 10


StopIteration: ignored

La première ligne de délégués principaux dans le générateur interne, et produit les valeurs, en les extrayant directement de celui-ci. Ce n'est pas nouveau, comme nous l'avons déjà vu. Remarquez, cependant, comment la fonction générateur sequence() renvoie la valeur finale, qui est assignée dans la première ligne à la variable nommée step1, et comment cette valeur est correctement utilisée au début de l'instance suivante de ce générateur.

Au final, cet autre générateur renvoie également la deuxième valeur finale (10), et le générateur principal, à son tour, renvoie la somme de celles-ci (5+10=15), qui est la valeur que nous voyons une fois l'itération arrêtée

    Nous pouvons utiliser yield from pour capturer la dernière valeur d'une
    coroutine après qu'elle ait terminé son traitement.

Avec cet exemple et ceux présentés dans la section précédente, vous pouvez avoir une idée de ce que fait le rendement de la construction en Python. Le rendement de la construction prendra le générateur et transmettra son itération en aval, mais une fois terminé, il interceptera son exception StopIteration, en obtiendra la valeur et renverra cette valeur à la fonction appelante. L'attribut value de l'exception StopIteration devient le résultat de l'instruction.

C'est une construction puissante, car en conjonction avec le sujet de la section suivante (comment envoyer et recevoir des informations contextuelles d'un sous-générateur), cela signifie que les coroutines peuvent prendre la forme de quelque chose de similaire aux threads.

## Envoi et réception de données vers et depuis un sous-générateur 

Nous allons maintenant voir l'autre fonctionnalité intéressante la syntaxe yield from, qui est probablement ce qui lui donne toute sa puissance. Comme nous l'avons déjà présenté lorsque nous avons exploré les générateurs agissant comme des coroutines, nous savons que nous pouvons envoyer des valeurs et leur lancer des exceptions, et, dans de tels cas, la coroutine recevra soit la valeur pour son traitement interne, soit elle devra gérer le exception en conséquence.

Si nous avons maintenant une coroutine qui délègue dans d'autres (comme dans l'exemple précédent), nous aimerions également conserver cette logique. Devoir le faire manuellement serait assez complexe (vous pouvez jeter un coup d'œil au code décrit dans PEP-380 si nous n'avions pas géré cela par yield from automatiquement)

Pour illustrer cela, gardons le même générateur de niveau supérieur (main) tel quel par rapport à l'exemple précédent (appelant d'autres générateurs internes), mais modifions les générateurs internes pour les rendre capables de recevoir des valeurs et de gérer des exceptions. Le code n'est probablement pas idiomatique, uniquement dans le but de montrer comment fonctionne ce mécanisme :

In [ ]:
class CustomException(Exception):
    """A type of exception that is under control."""


def sequence(name, start, end):
    value = start
    print("%s started at %i", name, value)
    while value < end:
        try:
            received = yield value
            print("%s received %r", name, received)
            value += 1
        except CustomException as e:
            print("%s is handling %s", name, e)
            received = yield "OK"
    return end


def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)
    return step1 + step2

Maintenant, nous allons appeler la coroutine principale, non seulement en l'itérant, mais aussi en lui fournissant des valeurs et en lançant des exceptions afin de voir comment elles sont gérées à l'intérieur de la séquence :

In [ ]:
g = main()
next(g)

%s started at %i first 0


0

In [ ]:
next(g)

%s received %r first None


1

In [ ]:
g.send("value for 1")

%s received %r first value for 1


2

In [ ]:
g.throw(CustomException("controlled error"))

%s is handling %s first controlled error


'OK'

In [ ]:
g.throw(CustomException("exception at second generator"))

CustomException: ignored

Cet exemple nous dit beaucoup de choses différentes. Remarquez comment nous n'envoyons jamais de valeurs à sequence, mais uniquement à main, et même ainsi, le code qui reçoit ces valeurs est les générateurs imbriqués. Même si nous n'envoyons jamais explicitement quoi que ce soit à la séquence, elle reçoit les données au fur et à mesure qu'elles sont transmises par yield from.

La coroutine principale appelle deux autres coroutines en interne, produisant leurs valeurs, et elle sera suspendue à un moment donné dans l'une d'entre elles. Lorsqu'il est arrêté au premier, nous pouvons voir les logs nous indiquant que c'est cette instance de la coroutine qui a reçu la valeur que nous avons envoyée. La même chose se produit lorsque nous lui lançons une exception. Lorsque la première coroutine se termine, elle renvoie la valeur qui a été assignée dans la variable nommée step1, et passée en entrée pour la deuxième coroutine, qui fera de même (elle gérera les appels send() et throw(), en conséquence )

Il en va de même pour les valeurs produites par chaque coroutine. Lorsque nous sommes à une étape donnée, le retour de l'appel de send() correspond à la valeur que la sous-coroutine (celle à laquelle main est actuellement suspendu) a produit. Lorsque nous lançons une exception en cours de traitement, la coroutine de séquence produit la valeur OK, qui est propagée à la coroutine appelée (main) et qui, à son tour, aboutira à l'appelant de main. Comme prévu, ces méthodes, ainsi que le rendement de , nous offre de nombreuses nouvelles fonctionnalités (quelque chose qui peut ressembler à des threads). Cela ouvre les portes à la programmation asynchrone, que nous explorerons ensuite

## Programmation asynchrone

Avec les constructions que nous avons vues jusqu'ici, nous pouvons créer des programmes asynchrones en Python. Cela signifie que nous pouvons créer des programmes qui ont de nombreuses coroutines, les programmer pour qu'elles fonctionnent dans un ordre particulier et basculer entre elles lorsqu'elles sont suspendues après qu'un rendement de a été appelé sur chacune d'elles.

Le principal avantage que nous pouvons en tirer est la possibilité de paralléliser les opérations d'E/S de manière non bloquante. Ce dont nous aurions besoin, c'est d'un générateur de bas niveau (généralement implémenté par une bibliothèque tierce) qui sache comment gérer les E/S réelles pendant que la coroutine est suspendue. L'idée est que la coroutine effectue une suspension afin que notre programme puisse gérer une autre tâche en attendant. 

L'application récupérerait le contrôle au moyen de l'instruction yield from, qui suspendra et produira une valeur pour l'appelant (comme dans les exemples que nous avons vus précédemment lorsque nous avons utilisé cette syntaxe pour modifier le flux de contrôle du programme )

C'est à peu près la façon dont la programmation asynchrone fonctionnait en Python depuis quelques années, jusqu'à ce qu'il soit décidé qu'un meilleur support syntaxique était nécessaire

Le fait que les coroutines et les générateurs soient techniquement les mêmes provoque une certaine confusion. Syntaxiquement (et techniquement), ils sont identiques, mais sémantiquement, ils sont différents. Nous créons des générateurs lorsque nous voulons réaliser une itération efficace. 

Nous créons généralement des coroutines dans le but d'exécuter des opérations d'E/S non bloquantes. Bien que cette différence soit claire, la nature dynamique de Python permettrait toujours aux développeurs de mélanger ces différents types d'objets, se retrouvant avec une erreur d'exécution très tardivement. étape du programme. 

Rappelez-vous que dans la forme la plus simple et la plus basique du rendement de la syntaxe, nous avons utilisé cette construction sur des objets itérables (nous avons créé une sorte de fonction de chaîne appliquée sur des chaînes, des listes, etc.). Aucun de ces objets n'était des coroutines, et cela fonctionnait toujours. Ensuite, nous avons vu que nous pouvions avoir plusieurs coroutines, utiliser yield from pour envoyer la valeur (ou les exceptions) et récupérer des résultats. Ce sont clairement deux cas d'utilisation très différents ; cependant, si nous écrivons quelque chose dans le sens de la déclaration suivante

In [ ]:
result = yield from iterable_or_awaitable()

Ce que iterable_or_awaitable renvoie n'est pas clair. Il peut s'agir d'un simple itérable, tel qu'une chaîne, et sa syntaxe peut toujours être correcte. Ou, il pourrait s'agir d'une coroutine réelle. Le coût de cette erreur sera payé bien plus tard, au moment de l'exécution. Pour cette raison, le système de typage en Python a dû être étendu. Avant Python 3.5, les coroutines n'étaient que des générateurs avec un décorateur @coroutine appliqué, et elles devaient être appelées avec le yeld from de la syntaxe. Maintenant, il existe un type d'objet spécifique que l'interpréteur Python reconnaît comme tel, c'est-à-dire une coroutine

Ce changement annonçait également des changements de syntaxe. La syntaxe await et async def  a été introduite. Le premier est destiné à être utilisé à la place de yield from, et il ne fonctionne qu'avec des objets en attente (ce que les coroutines sont commodément). Essayer d'appeler await avec quelque chose qui ne respecte pas l'interface d'un waitable lèvera une exception (c'est un bon exemple de la façon dont les interfaces peuvent aider à obtenir une conception plus solide, en évitant les erreurs d'exécution)

async def est la nouvelle façon de définir les coroutines, remplaçant le décorateur susmentionné, et cela crée en fait un objet qui, lorsqu'il est appelé, renverra une instance d'une coroutine. De la même manière que lorsque vous invoquez une fonction génératrice, l'interpréteur vous renverra un objet générateur, lorsque vous invoquez un objet défini avec async def, il vous donnera un objet coroutine qui a une méthode __await__, et donc peut être utilisé dans les expressions d'attente.

Sans entrer dans tous les détails et possibilités de la programmation asynchrone en Python, on peut dire que malgré la nouvelle syntaxe et les nouveaux types, cela ne fait rien de fondamentalement différent des concepts que nous avons abordés dans ce chapitre.

L'idée derrière la programmation asynchrone en Python est qu'il existe une boucle d'événements (généralement asynchrone car c'est celle qui est incluse dans la bibliothèque standard, mais il y en a beaucoup d'autres qui fonctionneront de la même manière) qui gère une série de coroutines. Ces coroutines appartiennent à la boucle d'événements, qui va les appeler selon son mécanisme d'ordonnancement. Lorsque chacune de ces exécutions, il appellera notre code (selon la logique que nous avons définie à l'intérieur de la coroutine que nous avons programmée), et lorsque nous voulons reprendre le contrôle de la boucle d'événements, nous appelons await <coroutine>, qui traitera une tâche de manière asynchrone. La boucle d'événements reprendra et une autre coroutine aura lieu pendant que cette opération est en cours d'exécution. Ce mécanisme représente les bases du fonctionnement de la programmation asynchrone en Python. Vous pouvez penser que la nouvelle syntaxe ajoutée pour les coroutines (async def / await) n'est qu'une API vous permettant d'écrire du code d'une manière qui sera appelée par la boucle d'événement. Par défaut, cette boucle d'événement sera généralement asynchrone car c'est celle qui vient dans la bibliothèque standard, mais tout système de boucle d'événement qui correspond à l'API fonctionnera. Cela signifie que vous pouvez utiliser des bibliothèques comme uvloop (https://github.com/MagicStack/uvloop) et trio (https://github.com/python-trio/trio), et le code fonctionnera de la même manière. Vous pouvez même enregistrer votre propre boucle d'événements, et cela devrait également fonctionner de la même manière (à condition de respecter l'API, c'est-à-dire). En pratique, il existe davantage de particularités et de cas extrêmes qui dépassent le cadre de ce livre. Il convient cependant de mentionner que ces concepts sont liés aux idées introduites dans ce chapitre et que cette arène est un autre endroit où les générateurs démontrent qu'ils sont un concept central du langage, car il y a beaucoup de choses construites dessus.